<a href="https://colab.research.google.com/github/A-Janj/Computational_Pathology_Preprocessing/blob/main/TCGA_BRCA_pathology_thumbnails.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("Bismillah")

Bismillah


# **Installations**

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 212.4 MB 59 kB/s 
     |████████████████████████████████| 198 kB 55.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=12c9d58a978df6c5213d80bb5365dfde4c5ceecfdcb656d63de0ecdd1a463f53
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 40 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Rea

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Let's import the libraries we will need
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# %matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
import re

In [ ]:
!sudo add-apt-repository ppa:webupd8team/java
!sudo apt-get update
!sudo apt-get install oracle-java8-installer

 The Oracle JDK License has changed for releases starting April 16, 2019.

The new Oracle Technology Network License Agreement for Oracle Java SE is substantially different from prior Oracle JDK licenses. The new license permits certain uses, such as personal use and development use, at no cost -- but other uses authorized under prior Oracle JDK licenses may no longer be available. Please review the terms carefully before downloading and using this product. An FAQ is available here: https://www.oracle.com/technetwork/java/javase/overview/oracle-jdk-faqs.html

Oracle Java downloads now require logging in to an Oracle account to download Java updates, like the latest Oracle Java 8u211 / Java SE 8u212. Because of this I cannot update the PPA with the latest Java (and the old links were broken by Oracle).

For this reason, THIS PPA IS DISCONTINUED.

For Oracle Java 11, see a different PPA -> https://www.linuxuprising.com/2019/06/new-oracle-java-11-installer-for-ubuntu.html

That same PPA a

In [ ]:
# # Import and create a new SQLContext 
# from pyspark.sql import SQLContext

# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()
sqlContext = SQLContext(sc)

# **Pre-Processing**

In [ ]:
from pyspark.sql import SparkSession
#Create PySpark SparkSession
spark = SparkSession.builder \
    .master("local[1]") \
    .appName("SparkByExamples.com") \
    .getOrCreate()


In [ ]:
tcga_file_path = "/content/drive/MyDrive/AIDPath/TCGA-BRCA Data1.csv"
thumbnails_path = "/content/drive/MyDrive/AIDPath/Thubnail_list.csv"

import pandas as pd
thumnails_list_panda = pd.read_csv(thumbnails_path)
thubnail_list = thumnails_list_panda["0"].tolist()
# print(type(thumnails_list_panda))

tcga_table = spark.read.csv(tcga_file_path, header= True)
tcga_table.printSchema()
tcga_table.show()
tcga_table.count()

root
 |-- id: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- age_at_initial_pathologic_diagnosis: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- race: string (nullable = true)
 |-- ajcc_pathologic_tumor_stage6: string (nullable = true)
 |-- ajcc_pathologic_tumor_stage7: string (nullable = true)
 |-- clinical_stage: string (nullable = true)
 |-- histological_type: string (nullable = true)
 |-- histological_grade: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: string (nullable = true)
 |-- _c18: string (nullable = true)
 |-- _c19: string (nullable = true)
 |-- _c20: string (nullable = true)
 |-- _c21: string (nullable = true)
 |-- _c22: string (nullable = true)
 |-- _c23: string (nullable = true)
 |-- _c24: stri

1097

In [ ]:
def match_with_thumbnail_list(x):
  strings_with_substring = [string for string in thubnail_list if x in string]
  if len(strings_with_substring) == 0:
    returning = ""
  else:
    returning = strings_with_substring[0]
  if len(strings_with_substring) > 1: # If more than one is returned then I wanna know
    print(len(strings_with_substring))
  return returning

func_udf = udf(match_with_thumbnail_list)  # pyspark.sql.functions include udf()

tcga_table = tcga_table.withColumn("original_thumnail_names", func_udf('Name'))
tcga_table.show()
tcga_table.count()

+---+------------+----+-----------------------------------+------+--------------------+----------------------------+----------------------------+--------------+--------------------+------------------+-----+----------+----+----+------------------+----+----+----+---------------+---------+----+----+----+----+----+----+----+----+----+----+-----------------------+
| id|        Name|Type|age_at_initial_pathologic_diagnosis|gender|                race|ajcc_pathologic_tumor_stage6|ajcc_pathologic_tumor_stage7|clinical_stage|   histological_type|histological_grade| _c11|      _c12|_c13|_c14|              _c15|_c16|_c17|_c18|           _c19|     _c20|_c21|_c22|_c23|_c24|_c25|_c26|_c27|_c28|_c29|_c30|original_thumnail_names|
+---+------------+----+-----------------------------------+------+--------------------+----------------------------+----------------------------+--------------+--------------------+------------------+-----+----------+----+----+------------------+----+----+----+---------------

1097

In [ ]:
cancer_stage_list = tcga_table.select("ajcc_pathologic_tumor_stage6").distinct().rdd.map(lambda r: r[0]).collect()
cancer_stage_list

['Stage IIIB',
 'Stage III',
 'Stage IIB',
 'Stage IIIA',
 'Stage IV',
 'Stage X',
 '[Not Available]',
 'Stage IIIC',
 '[Discrepancy]',
 'Stage IB',
 'Stage I',
 'Stage IA',
 'Stage II',
 'Stage IIA']

In [ ]:
cancer_substage_list = tcga_table.select("ajcc_pathologic_tumor_stage7").distinct().rdd.map(lambda r: r[0]).collect()
cancer_substage_list

['Other, specify',
 'Medullary Carcinoma',
 'Mixed Histology (please specify)',
 '[Not Available]',
 'Infiltrating Carcinoma NOS',
 'Infiltrating Ductal Carcinoma',
 'Metaplastic Carcinoma',
 'Infiltrating Lobular Carcinoma',
 'Mucinous Carcinoma']

# Saving TCGA list with respect to TNM stage and Subtype

In [ ]:
for typeCancer in cancer_stage_list:
  for subtype in cancer_substage_list:
    temp_table = tcga_table.select("id", "original_thumnail_names", "Name", "ajcc_pathologic_tumor_stage6", "ajcc_pathologic_tumor_stage7").filter((tcga_table["ajcc_pathologic_tumor_stage6"] == typeCancer) & (tcga_table["ajcc_pathologic_tumor_stage7"] == subtype)  & (tcga_table.original_thumnail_names != ""))
    if temp_table.count() > 0:
      save_path= "/content/drive/MyDrive/AIDPath/Stages_categorized_thumbnails/"+typeCancer+"_"+subtype+".csv"
      temp_table.toPandas().to_csv(save_path)